In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../../pooling/data/MEX2/")#[0:25]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

1779 municipalities.


In [6]:
# image_names

In [7]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .001)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .60,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [9]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [10]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [13]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  1067
Num validation:  712


In [14]:
# for i,o in train_dl:
#     print(i)
# #     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch:  0
  Training Loss:  1703.9366634390346
  Validation Loss:  1651.0035135933522


Epoch:  1
  Training Loss:  1650.6986408609034
  Validation Loss:  1647.8451146758005


Epoch:  2
  Training Loss:  1627.2687188017112
  Validation Loss:  1798.0200579353932


Epoch:  3
  Training Loss:  1624.4881618110064
  Validation Loss:  1661.0538999578926


Epoch:  4
  Training Loss:  1612.9354908164685
  Validation Loss:  1655.3171787476272
VALS BEFORE:  [[0, 0, 0], [-14.076246334310852, 0, 0], [100.0, -8761.728395061727, 0], [0, 0, 0], [0, 100.0, -446.70658682634735], [28.622832842656198, -307.0572569906791, 3.717948717948718], [0, 0, 100.0], [100.0, 0, 0], [0, 0, 0], [100.0, 0, 0], [0, 100.0, -187.6793341364582], [36.4168618266979, 0, 100.0], [0, 0, 0], [100.0, 0, 100.0], [0, 0, 0], [96.66361974405851, 0, 0], [-846.2962962962964, 96.2369337979094, 0], [57.761732851985556, 88.67075664621676, -12125.0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1131.4285714285716, 88.78205128205127, 16.35

In [ ]:
butler.image_sizes